In [1]:
import torch

## 自动求导

### 假设我们想对函数$y=2\pmb{\text{x}}^T\pmb{\text{x}}$关于列向量$\pmb{\text{x}}$求导

In [2]:
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

### 在我们计算$y$关于$\pmb{\text{x}}$的梯度之前，我们需要一个地方来存储梯度

In [3]:
x.requires_grad_(True) # 等价于 x = torch.arange(4.0, requires_grad_=True)
x.grad # 默认值是None

### 现在让我们计算$y$

In [4]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

### 通过调用反向传播函数来自动计算$y$关于$\pmb{\text{x}}$每个分量的梯度

In [5]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [6]:
x.grad == 4 * x

tensor([True, True, True, True])

### 再计算$\pmb{\text{x}}$的另一个函数之前需要清除之前的梯度，因为在默认情况下，pytorch会累积梯度

In [8]:
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

### 深度学习中，我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和

In [10]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

### 将某些计算移动到记录的计算图之外，这在以后把网络参数固定住的时候是一个很有用的功能

In [11]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [12]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

### 即使构建函数的计算图需要通过python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度

In [13]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [14]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

In [15]:
a.grad == d / a

tensor(True)

In [ ]:
# 调用f(a)函数返回的值，是一个关于a的线性函数，所以d关于a的导数就是线性函数的系数